In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [ ]:
IMG_HEIGHT = 32
IMG_WIDTH = 32
BATCH_SIZE = 32
EPOCHS = 30
NUM_CLASSES = 59
DATA_DIR = "data_split"

In [ ]:
def prepare_data():
    train_gen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

    val_test_gen = ImageDataGenerator(rescale=1./255)

    train_set = train_gen.flow_from_directory(
        os.path.join(DATA_DIR, "train"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical"
    )

    val_set = val_test_gen.flow_from_directory(
        os.path.join(DATA_DIR, "val"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical"
    )

    test_set = val_test_gen.flow_from_directory(
        os.path.join(DATA_DIR, "test"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )

    return train_set, val_set, test_set

In [ ]:
def build_vgg(num_classes, fine_tune=False):
    base_model = VGG16(weights="imagenet", include_top=False,
                       input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    if fine_tune:
        # Freeze only first 15 layers, train rest
        for layer in base_model.layers[:15]:
            layer.trainable = False
        for layer in base_model.layers[15:]:
            layer.trainable = True
    else:
        # Freeze all layers
        for layer in base_model.layers:
            layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    lr = 1e-5 if fine_tune else 1e-4
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
def plot_history(history, title="Training and Validation Loss"):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:

def main():
    train_set, val_set, test_set = prepare_data()

    # Stage 1: Train with frozen VGG16
    print("\n--- Training with frozen VGG16 ---")
    model = build_vgg(NUM_CLASSES, fine_tune=False)
    history = model.fit(train_set, validation_data=val_set,
                        epochs=EPOCHS,
                        callbacks=[EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)])
    print("Final test accuracy (frozen):", model.evaluate(test_set)[1])
    print(classification_report(test_set.classes,
                                np.argmax(model.predict(test_set), axis=1),
                                target_names=list(test_set.class_indices.keys())))
    model.save("vgg16_frozen.keras")
    plot_history(history, title="VGG16 (Frozen) Loss")

    # Stage 2: Fine-tune last conv blocks
    print("\n--- Fine-tuning VGG16 ---")
    ft_model = build_vgg(NUM_CLASSES, fine_tune=True)
    history_ft = ft_model.fit(train_set, validation_data=val_set,
                              epochs=EPOCHS,
                              callbacks=[EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)])
    print("Final test accuracy (fine-tuned):", ft_model.evaluate(test_set)[1])
    print(classification_report(test_set.classes,
                                np.argmax(ft_model.predict(test_set), axis=1),
                                target_names=list(test_set.class_indices.keys())))
    #ft_model.save("vgg16_finetuned.keras")
    plot_history(history_ft, title="VGG16 (Fine-tuned) Loss")

In [ ]:
if __name__ == "__main__":
    main()